In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
encoder = LabelEncoder()
OHE = OneHotEncoder(sparse=False)

In [ ]:
dados = pd.read_csv('../input/hotel-booking-demand/hotel_bookings.csv')
dados.head()

## Preparação dos dados / Preparing and transforming data ##

In [ ]:
ax, fig = plt.subplots(figsize=(16,8))
sns.heatmap(dados.corr(), annot=True)

In [ ]:
dados.isnull().sum()

In [ ]:
dados['country'].fillna(dados['country'].mode()[0], inplace = True)
dados['agent'].fillna(dados['agent'].mode()[0], inplace = True)
dados['company'].fillna(dados['company'].mode()[0], inplace = True)
dados['children'].fillna(dados['children'].mode()[0], inplace=True)

In [ ]:
binarizer = LabelBinarizer()
dados['hotel'] = binarizer.fit_transform(dados['hotel'])

In [ ]:
dados['reservation_status_date'] = pd.to_datetime(dados['reservation_status_date'], format = "%Y-%m-%d")

In [ ]:
dados['reservation_status_date'] = dados['reservation_status_date'].apply(lambda x: x.strftime("%Y-%m"))

In [ ]:
dados.set_index("reservation_status_date", inplace=True)

In [ ]:
dados.head()

In [ ]:
remover1 = ['reserved_room_type','assigned_room_type','reservation_status']
remover2 = ['arrival_date_month','meal','market_segment','distribution_channel','deposit_type','customer_type']
remover2 = np.array(remover2)

In [ ]:
for rem in remover1:
  dados[rem] = encoder.fit_transform(dados[rem])

In [ ]:
for rem2 in remover2:
  dados = pd.get_dummies(dados)

In [ ]:
dados.head()

In [ ]:
dados.groupby(dados.index).is_canceled.sum().plot(kind='line', figsize=(12,6), xlabel="Período", ylabel='Número de Cancelamentos', title='Número de Cancelamentos de Hospedagem ao Mês', grid=True)

In [ ]:
dados.groupby(dados.index).lead_time.sum().plot(kind='line', figsize=(12,6), xlabel="Período", ylabel='Número de Cancelamentos', title='Número de Cancelamentos de Hospedagem ao Mês', grid=True)

## Criando modelo / Creating model ##

In [ ]:
x = dados.drop("is_canceled", 1)
y = dados.loc[:,['is_canceled']]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
x_test, x_train, y_test, y_train = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
modelo = RandomForestClassifier()
modelo.fit(x_train, y_train)

In [ ]:
predicao = modelo.predict(x_test)

In [ ]:
print(classification_report(y_test, predicao))

In [ ]:
print(confusion_matrix(y_test, predicao))

In [ ]:
print(accuracy_score(y_test, predicao))

### Devido ao elevado grau do coeficiente de predição, acima de 0.99, pode-se deduzir que o modelo sofre de sobreajustamento, assim sendo, deve-se re-analizar as váriaveis utilizadas ###

### Model looks overfitted ###

In [ ]:
dados2 = pd.read_csv('../input/hotel-booking-demand/hotel_bookings.csv')
dados2.head()

In [ ]:
dados2.groupby(['is_canceled']).sum()

In [ ]:
dados2.groupby('reservation_status').sum()

In [ ]:
x2 = dados.drop(['is_canceled', 'reservation_status'], axis = 1)

In [ ]:
x_test, x_train, y_test, y_train = train_test_split(x2, y, test_size=0.2, random_state=0)

In [ ]:
modelo2 = RandomForestClassifier()
modelo2.fit(x_train, y_train)

In [ ]:
predicao = modelo2.predict(x_test)

In [ ]:
print(classification_report(y_test, predicao))

In [ ]:
print(confusion_matrix(y_test, predicao))